# Testing VAI-Utils LSTM Timeseries 

We run through the `learn-time` flow to verify the code runs. 

Showcases local training using aggregated features only. 

* TODO: move `config["hid"]` to load_data code 
* TODO: implement+test with preexisting x features/no aggregation

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
import importlib
import numpy as np

sys.path.append("../src")
# setting path
from vai_utils import models, data_process, preprocess

In [ ]:
# datapath for testing
DATA_PATH = 'data/random_no_target'

In [ ]:
nld = [
        (1, 1, 2022), (1, 17, 2022), (2, 21, 2022),
        (5, 30, 2022), (7, 4, 2022), (9, 5, 2022), 
        (10, 10, 2022), (11, 11, 2022), (11, 24, 2022),
        (12, 26, 2022), 
        (1, 1, 2023), (1, 17, 2023), (2, 21, 2023),
        (5, 30, 2023), (7, 4, 2023), (9, 5, 2023), 
        (10, 10, 2023), (11, 11, 2023), (11, 24, 2023),
        (12, 26, 2023), 
    
    ]

In [ ]:
model_test_config =  {
        "x": {}, "y": {},
        "mid":"alan_test",
        "h_params": {
            "learning_rate": 0.01,
            "epochs": 2,
            "batch_size": 1000,
            "deep_lyrs": 1,
            "step": "hour",
            "optimizer": "adam",
            "tid": 1,
            "train_local": {"checkpoint_path":"models"}, # enables local training and saving
        },
        "metrics": ["mse"],
        "null": "infer",
        "timeseries": {
            "datetime": {
                "step": "hour",
                "time-window":("11:00", "22:00"),
                'remove_hours': True,
                "add_nld": 'auto',
                "delta": 8,
                "lag": 8,
                "steps_in":  11,  # 1 day == 11 hours
                "steps_out": 77,  # 7 days = 11 hours x 7 days
                "add_meanstd": True
            },
            "lstm": {
                "LSTM_layers": 2,
                "LSTM_nodes": 100,
                "LSTM_dropout": 0.1,
                "LSTM_batch_norm": True,   # changed to true
                "first_activation": "relu",
                "activation": "relu",
                "final_activation": "linear",
                "l1_l2": 0
            }
        }
    }

# Adds filepaths to config
file_paths = []
for root, dirs, files in os.walk(os.path.abspath("data/random_no_target")):
    for file in files:
        full_path = os.path.join(root, file)
        if 'csv' in full_path: file_paths.append(full_path)
model_test_config['file_paths'] = file_paths  

In [ ]:
# Create xy dataset without s3 
xy_dataset = data_process.load_data.dataframe(model_test_config, None, s3=None)

In [ ]:
# Make model save path. TODO: this should be in the code itself per config
model_test_config["hid"] = '_'.join([str(s) for s in model_test_config["h_params"].values() if type(s) != dict])
os.makedirs(f'{model_test_config["h_params"]["train_local"]["checkpoint_path"]}/{model_test_config["mid"]}/{model_test_config["hid"]}', exist_ok=True)

x, y, data_mappings = preprocess.datetime(xy_dataset, model_test_config)

In [ ]:
# Train the model based on the x, y values
models.train_lstm(x, y, model_test_config)

In [ ]:
data.html_to_json()
